- pattern matching
- fuzzy matching
- levenshtein distance

In [13]:
import pandas as pd
import psycopg2
from psycopg2 import sql # format sql queries with params
from config import config

# Get the config params
params_ = config()
# Connect to the Postgres_DB:
conn = psycopg2.connect(**params_)
# Create new_cursor allowing us to write Python to execute PSQL:
cur = conn.cursor()
conn.autocommit = True

In [8]:
car_dlr_one=pd.read_csv('data/cardealer_add_deduct_list.csv')
car_dlr_two=pd.read_csv('data/car_dealer_firstpart.csv')

car_dlr_two.head()

,publish_date,data_freq,state,country,vin,uvc,groupnum,model_year,make,model,...,num_gears,ext_doors,airbags,anti_corrosion_warranty,basic_warranty,powertrain_warranty,road_assist_warranty,add_deduct_list,model_number_list,engine_turbocharged
0,8/28/2020,D,NT,US,102280487,2019485072,4107,2019,Kia,Soul,...,6,4,Side Curtain,"5-year/100,000-mile, Limited","5-year/60,000-mile, Limited","10-year/100,000-mile, Limited","5-year/60,000-mile","[{'uoc': '8C', 'name': '18-Inch Wheels', 'xcle...",['B2522'],NaN
1,8/28/2020,D,NT,US,102298407,2017160869,141,2017,Chevrolet,Malibu,...,6,4,Side Curtain,"6-year/100,000-mile","3-year/36,000-mile, Limited","5-year/60,000-miles, Limited","5-year/60,000-mile","[{'uoc': '8C', 'name': '18-Inch Wheels', 'xcle...",['1ZD69'],Turbocharg
2,8/28/2020,D,NT,US,102633593,2017300901,16,2017,Ford,Fusion,...,6,4,Side Curtain,5-year/Unlimited-mile,"3-year/36,000-mile","5-year/60,000-mile","5-year/60,000-mile","[{'uoc': '8C', 'name': '18-Inch Al Whl', 'xcle...",['P0H'],NaN


In [9]:
def create_staging_table(cursor):
    cursor.execute("""
        DROP TABLE IF EXISTS car_dealer_01;
        CREATE UNLOGGED TABLE car_dealer_01 (
            vin                varchar(30),
            idx                int,
            model_number_list  text
            
        );""")
    
# Send the Schema to PSQL
with conn.cursor() as cursor:
    create_staging_table(cursor)

In [10]:
def create_staging_table(cursor):
    cursor.execute("""
        DROP TABLE IF EXISTS car_dealer_02;
        CREATE UNLOGGED TABLE car_dealer_02 (
            publish_date       date,
            vin                varchar(30),
            make               varchar(30),
            model              varchar(30),               
            model_year         int 
        );""")
    
# Send the Schema to PSQL
with conn.cursor() as cursor:
    create_staging_table(cursor)

# Information_schema

In [16]:
ss="""
SELECT table_name, column_name, data_type 
FROM information_schema.columns 
WHERE table_name={table_name} ;
"""
o = sql.SQL(ss).format(table_name = sql.Literal('car_dealer_02'))
cur.execute(o)
cur.fetchall()


[('car_dealer_02', 'publish_date', 'date'),
 ('car_dealer_02', 'vin', 'character varying'),
 ('car_dealer_02', 'make', 'character varying'),
 ('car_dealer_02', 'model', 'character varying'),
 ('car_dealer_02', 'model_year', 'integer')]

# Dataframe to PSQL

In [18]:
from psycopg2 import extras


def fcn(df,table,cur):
#     df=addr_df_
#     table='staging_fake_ppl02'
    # df is the dataframe
    if len(df) > 0:
        df_columns = list(df)
        # create (col1,col2,...)
        columns = ",".join(df_columns)

        # create VALUES('%s', '%s",...) one '%s' per column
        values = "VALUES({})".format(",".join(["%s" for _ in df_columns])) 

        #create INSERT INTO table (columns) VALUES('%s',...)
        insert_stmt = "INSERT INTO {} ({}) {}".format(table,columns,values)
        cur.execute("truncate " + table + ";")  #avoiding uploading duplicate data!
        cur = conn.cursor()
        psycopg2.extras.execute_batch(cur, insert_stmt, df.values)
    conn.commit()

In [22]:
# Select columns
car_dlr_subset_01=car_dlr_one.loc[:,['idx','vin','model_number_list']]
car_dlr_subset_01.values

array([[0, 102280487, "['B2522']"],
       [0, 102280487, "['B2522']"],
       [1, 102298407, "['1ZD69']"],
       [1, 102298407, "['1ZD69']"],
       [1, 102298407, "['1ZD69']"],
       [1, 102298407, "['1ZD69']"],
       [2, 102633593, "['P0H']"],
       [2, 102633593, "['P0H']"],
       [2, 102633593, "['P0H']"],
       [2, 102633593, "['P0H']"],
       [2, 102633593, "['P0H']"],
       [2, 102633593, "['P0H']"]], dtype=object)

In [23]:
# Convert Dataframe to PSQL
fcn(car_dlr_subset_01,'car_dealer_01',cur)
cur.execute("select * from car_dealer_01 limit 4")
cur.fetchall()


[('102280487', 0, "['B2522']"),
 ('102280487', 0, "['B2522']"),
 ('102298407', 1, "['1ZD69']"),
 ('102298407', 1, "['1ZD69']")]

In [24]:
# Convert Dataframe to PSQL(2)
#Insert data
car_dlr_subset02=car_dlr_two.loc[:,['publish_date','vin','make','model','model_year']]
fcn(car_dlr_subset02,'car_dealer_02',cur)

cur.execute("select * from car_dealer_02 limit 4")
cur.fetchall()

[(datetime.date(2020, 8, 28), '102280487', 'Kia', 'Soul', 2019),
 (datetime.date(2020, 8, 28), '102298407', 'Chevrolet', 'Malibu', 2017),
 (datetime.date(2020, 8, 28), '102633593', 'Ford', 'Fusion', 2017)]

# Convert Column Data-type

In [26]:
ww="""
ALTER TABLE car_dealer_02 ALTER COLUMN vin TYPE integer USING (vin::integer);
"""
cur.execute(ww)

In [27]:
ss ="""
SELECT table_name, column_name, data_type
FROM information_schema.columns
WHERE table_name={table_name};"""

o = sql.SQL(ss).format(table_name = sql.Literal('car_dealer_02'))
cur.execute(o)
cur.fetchall()

[('car_dealer_02', 'publish_date', 'date'),
 ('car_dealer_02', 'vin', 'integer'),
 ('car_dealer_02', 'make', 'character varying'),
 ('car_dealer_02', 'model', 'character varying'),
 ('car_dealer_02', 'model_year', 'integer')]

# Convert to lowercase()

In [39]:
lwrcs='''
select first_name,lower(first_name) from staging_fake_ppl limit 4
'''

cur.execute(lwrcs)
pd.DataFrame(cur.fetchall(),columns=['Orginal_FirstName','Converted'])

,Orginal_FirstName,Converted
0,Donyell Ann,donyell ann
1,Bishop,bishop
2,Connor,connor
3,Kylie,kylie


# Mogrify
- 

In [28]:
# good for error checking or how exactly psql are interpreting what you input
print(cur.mogrify(o))
print('-------------------')
print(cur.mogrify(ww))

b"\nSELECT table_name, column_name, data_type\nFROM information_schema.columns\nWHERE table_name='car_dealer_02';"
-------------------
b'\nALTER TABLE car_dealer_02 ALTER COLUMN vin TYPE integer USING (vin::integer);\n'


# Fuzzy Match
- pg_trgm extension: fuzzy string matching
    - Find a person but you don't remember the name, only that it is close to Aldo
    - Find a person with last name similar to 'Bor'. Confidence > 0.4
- fuzzystrmatch extension: convert your string into four letter codes to search

# pg_trgm(1)

In [29]:
# pg_trgm will allow some fuzzy string matching
f = '''
CREATE EXTENSION pg_trgm;
SELECT * FROM staging_fake_ppl
ORDER BY SIMILARITY(first_name, 'Aldo') DESC
LIMIT 5;'''
cur.execute(f)
cur.fetchall()

# cur.execute('''Drop extension pg_trgm ''')

[('3118-3897-1674-7996', 'ajfeqi@iufeb.efk', 'Alfredo', 'Munoz', '5244214267'),
 ('9997-7751-7665-6525', 'hfl@bnkxsvp.ufq', 'Ali', 'Borrego', '2493418317'),
 ('1806-2704-9336-8126', 'swimxj@pjboqc.fow', 'Ali', 'Kouns', '4218255468'),
 ('2117-2588-7985-4527', 'dxbmswr@o.mlb', 'Alec', 'Johnson', '4259785921'),
 ('1758-6471-8929-2781', 'ruft@ip.drj', 'Alex', 'Acosta', '8728626214')]

In [34]:
# Alterate way
f = sql.SQL('''
CREATE EXTENSION IF NOT EXISTS pg_trgm;
SELECT * FROM staging_fake_ppl
ORDER BY SIMILARITY({first_name},{name})DESC
LIMIT 5;''').format(table_ = sql.Identifier('staging_fake_ppl'),
                    first_name = sql.Identifier('first_name'),
                    name =sql.Literal('Aldo'))
cur.execute(f)
cur.fetchall()

# cur.execute('''Drop extension pg_trgm ''')

[('3118-3897-1674-7996', 'ajfeqi@iufeb.efk', 'Alfredo', 'Munoz', '5244214267'),
 ('9997-7751-7665-6525', 'hfl@bnkxsvp.ufq', 'Ali', 'Borrego', '2493418317'),
 ('1806-2704-9336-8126', 'swimxj@pjboqc.fow', 'Ali', 'Kouns', '4218255468'),
 ('2117-2588-7985-4527', 'dxbmswr@o.mlb', 'Alec', 'Johnson', '4259785921'),
 ('1758-6471-8929-2781', 'ruft@ip.drj', 'Alex', 'Acosta', '8728626214')]

# pg_trgm(2)

In [35]:
conf='''
SELECT * FROM staging_fake_ppl
WHERE SIMILARITY(last_name,'Bor') > 0.4 ;

'''
cur.execute(conf)
cur.fetchall()

[('9538-4639-5634-2281', 'cvjkmi@t.cko', 'Cailinanne', 'Borja', '2968327286'),
 ('2285-2386-1324-5043', 'idv@pqdvscle.owa', 'Gunnar', 'Borda', '2595211562'),
 ('2731-8907-1044-6831', 'dstcjoaxw@uer.yef', 'Eric', 'Borom', '5864561645')]

In [36]:
#Alternative way
conf = sql.SQL('''
SELECT * FROM {table}
WHERE SIMILARITY({column},{name})>0.4;
''').format(table = sql.Identifier('staging_fake_ppl'),
            column = sql.Identifier('last_name'),
            name=sql.Literal('Bor'))
cur.execute(conf)
cur.fetchall()

[('9538-4639-5634-2281', 'cvjkmi@t.cko', 'Cailinanne', 'Borja', '2968327286'),
 ('2285-2386-1324-5043', 'idv@pqdvscle.owa', 'Gunnar', 'Borda', '2595211562'),
 ('2731-8907-1044-6831', 'dstcjoaxw@uer.yef', 'Eric', 'Borom', '5864561645')]

# fuzzystrmatch 

In [37]:

phone_='''
CREATE EXTENSION fuzzystrmatch;
SELECT*
FROM staging_fake_ppl WHERE
SOUNDEX(first_name) = SOUNDEX('Dony A');
'''
cur.execute(phone_)
cur.fetchall()

# Drop the extension if not needed anymore:
# cur.execute('''drop extension fuzzystrmatch''')

[('3388-9288-1720-9622', 'ysbe@ldajvymbqu.nga', 'Diana', 'Song', '6212544297'),
 ('6496-8623-1961-2916',
  'ktoey@vkqwexc.lbh',
  'Diana',
  'Olvedo Enriquez',
  '5234813586'),
 ('2725-5432-4694-7135', 'ivlm@wbsaim.lnj', 'Don', 'Martinez', '4657397958'),
 ('1736-4079-2112-1254', 'h@axlkjm.bip', 'Doan', 'Doht', '9416178251'),
 ('3929-1702-5794-9652', 'kofshc@cquxn.zxn', 'Deion', 'Holm', '4758758462'),
 ('3970-4084-4226-8551', 'fiw@uhfndrk.ugb', 'Danny', 'John', '2976273517'),
 ('7592-3801-8837-8387', 'o@bodzfaxin.dtg', 'Deanna', 'Newton', '7311849615'),
 ('7749-8107-5618-3026', 'ory@vukzhy.tnb', 'Danny', 'el-Zadeh', '6394522678'),
 ('9134-3311-6532-5063',
  'ynfhstlkwi@koayhrmx.kvd',
  'Danny',
  'Snively',
  '3174658291'),
 ('5067-7345-8241-7248',
  'ifshounla@mbkjcvewo.ezt',
  'Deeana',
  'Crespin',
  '6149173275'),
 ('9202-6310-9015-7438', 'cwxmht@luh.sft', 'Danni', 'Stahl', '9484396785'),
 ('2762-8722-9883-9155', 'c@rxtzgsemc.mnt', 'Deion', 'Begay', '4921596928'),
 ('9231-7927-1920-

# Levenshtein Distance

In [38]:
levi_='''
SELECT *,
LEVENSHTEIN(first_name, 'Dony w')
FROM staging_fake_ppl
ORDER BY LEVENSHTEIN(first_name, 'Dony w') ASC
LIMIT 5
'''

cur.execute(levi_)
cur.fetchall()

[('2958-5980-8949-4378', 'lrnyj@ianwzptm.vsy', 'Tony', 'Wu', '3948634793', 3),
 ('8966-6746-2369-5926', 'ln@xsjv.gsq', 'Rony', 'al-Taha', '5236191587', 3),
 ('2948-8505-3391-2567',
  'qoxdjanvfp@xqekfs.pqi',
  'Dondre',
  'Jaure',
  '9589588163',
  3),
 ('2725-5432-4694-7135',
  'ivlm@wbsaim.lnj',
  'Don',
  'Martinez',
  '4657397958',
  3),
 ('9231-7927-1920-8835',
  'mlotjsuy@asy.jou',
  'Don',
  "O'Leary",
  '7567296748',
  3)]

# Pattern Matching
- find first_name that are either starting with J or G  
- find vin numbers that match 102 and convert it to text
- find last name with [a letter] + ega


https://www.postgresql.org/docs/9.3/functions-matching.html

In [40]:
reg = '''
SELECT * 
FROM staging_fake_ppl 
WHERE first_name ~'^(J|G).*'
LIMIT 5;'''
cur.execute(reg)
cur.fetchall()

[('5909-1054-2911-2100',
  'nauvojf@tz.mjc',
  'Justin',
  'al-Yousif',
  '9214512756'),
 ('5027-1684-3271-9573',
  'esculkhwaj@lgime.kpd',
  'Justin',
  'Vigil',
  '4639124827'),
 ('4266-4647-9544-7225', 'bkzy@ozixylp.ixt', 'Jade', 'el-Qadir', '4121832691'),
 ('5737-9277-9259-1227',
  'vpzcje@jhtnqoaire.uag',
  'Juan',
  'Afuang',
  '8574182659'),
 ('6447-6210-9155-9438',
  'lwysbtujna@tsrzhyjm.wyp',
  'Joshua',
  'Brown',
  '3571528429')]

In [48]:
# LIKE 'a%' 
# LIKE 'abc'
# LIKE '_b_'
# % operator will match zero or more occurences
# _ the underscore will match single characters
reg2 = '''
SELECT * 
FROM car_dealer_02 
WHERE CAST(vin AS TEXT) LIKE '102%' 
'''
cur.execute(reg2)
cur.fetchall()

[(datetime.date(2020, 8, 28), 102280487, 'Kia', 'Soul', 2019),
 (datetime.date(2020, 8, 28), 102298407, 'Chevrolet', 'Malibu', 2017),
 (datetime.date(2020, 8, 28), 102633593, 'Ford', 'Fusion', 2017)]

In [49]:
lk_='''
select * from staging_fake_ppl where last_name like '_ega%'

'''
cur.execute(lk_)
df_=cur.fetchall()
             
# Get the Column Names:
cur.execute("Select * FROM staging_fake_ppl LIMIT 0")
colnames = [desc[0] for desc in cur.description]

pd.DataFrame(df_,columns=colnames).head()

,credit_card,email,first_name,last_name,primary_phone
0,1270-8146-6857-9420,pdw@uwarlpyvgj.unh,Umaima,Begay,5129433167
1,8233-3553-7350-4143,aqucnfhz@vtef.vcu,Santee,Vega,4362355976
2,6192-4667-4124-8778,woefqdgtmv@pqmxzscfg.svu,Theophilus,Begay,6286151359
3,7927-4545-1401-6815,hqvixtof@nj.ntf,Jordyn,Begaye,1258755914
4,2762-8722-9883-9155,c@rxtzgsemc.mnt,Deion,Begay,4921596928


psql shorcuts

https://www.postgresqltutorial.com/psql-commands/

Casting

https://www.postgresqltutorial.com/postgresql-cast/

https://stackoverflow.com/questions/56260366/insert-into-table-with-type-casting-and-condition-in-insert-statement

https://joshuaotwell.com/copy-and-cast-bulk-uploads-in-postgresql/

DATA Load

http://www.postgis.us/presentations/PGOpen2018_data_loading.pdf

